In [42]:
# -*- coding: utf-8 -*-
"""
Created on Sun Sep  6 15:59:28 2020

@author: Lyle Regenwetter (regenwet@mit.edu)
"""

import csv
import os.path
from os import path
import numpy as np
import time
import sys

def loadCSV():
    enableProgressBar=1
    #Progress Bar code
    if enableProgressBar==1:
        toolbar_width = 40
        sys.stdout.write("[%s]" % (" " * toolbar_width))
        sys.stdout.flush()
        sys.stdout.write("\b" * (toolbar_width+1)) # return to start of line, after '['
    
    #Initializations
    featureDict={}
    numFeatures=0
    numModels=4800
    csvUnassigned=1
    anomalyLog=''
    # modelIncrement=0
    
    #Loop over models
    for modelNum in range(1,numModels):
        csvPath='..\\CSVs\\'
        csvName=csvPath+"(" + str(modelNum) + ").csv"
        rowCount=0
        if path.exists(csvName): #If file exists
            with open(csvName, newline='', encoding="utf8") as csvFile:
                readCsv=csv.reader(csvFile, delimiter=',')
                for row in readCsv: #Loop over rows of CSV
                    if (rowCount>1):
    
                        #Debugging catch
    #                     if numModels==1036:
    #                         print(row)
                        
                        skip=0
                        
                        #Anomaly catch #0
                        if (len(row)>7 or len(row)<=2) and skip==0:
                            skip=1
                            row=['']*(6)
                            anomalyLog=anomalyLog + "Catch 0 on model " + str(modelNum) + " on row " + str(rowCount) + "\n"    
                           
                        #Anomaly catch #1
                        if (row[2]=="Driving" or row[2]=="Driven") and skip==0:
                            skip=1
                            anomalyLog=anomalyLog + "Catch 1 on model " + str(modelNum) + " on row " + str(rowCount) + "\n"
                        #Anomaly catch #2
                        if len(row)<6 and skip==0:
                            row=row+(['']*(6-len(row)))
                            anomalyLog=anomalyLog + "Catch 2 on model " + str(modelNum) + " on row " + str(rowCount) + "\n"    
                        #Anomaly catch #3
                        if len(row)==7 and skip==0:
                            row[3:5]=row[4:6]
                            del row[6]
                            anomalyLog=anomalyLog + "Catch 3 on model " + str(modelNum) + " on row " + str(rowCount) + "\n"    
                          
                        feature=tuple([row[0], row[1]])                           
                        if feature in featureDict and skip==0: #If particular feature has been seen before
                            featureIndex=featureDict[feature]
                            while (len(csvData[featureIndex])<modelNum): #Fill in blanks until index maches current
                                csvData[featureIndex].append(['','','',''])
                            if len(csvData[featureIndex])==modelNum: #Ensure Duplicate features in a model arent rerecorded
                                vals=[row[2], row[3], row[4], row[5]]
                                csvData[featureIndex].append(vals)
    
                        elif skip==0: #If particular feature has never been encountered
                            featureDict[feature]=numFeatures
                            featureIndex=numFeatures
                            numFeatures+=1
                            empty=np.full([modelNum-1,4], '').tolist() #Need to add empty cells for all previous models
                            empty.append([row[2], row[3], row[4], row[5]])
                            
                            if csvUnassigned: #If this is the absolute first entry
                                csvData=[empty]
                                csvUnassigned=0
                            else:
                                csvData.append(empty)     
                    rowCount+=1
    
    
    #More progress bar code
            if enableProgressBar==1:
                sys.stdout.write("-")
                sys.stdout.flush()
    if enableProgressBar==1:
        sys.stdout.write("]\n")
        
    for i in range(len(csvData)):
#         print(len(csvData[i]))
#         print(numModels-len(csvData[i]))
        empty=np.full([numModels-len(csvData[i]),4], '').tolist()
        csvData[i].extend(empty)
        
        csvData[i]=[(list(featureDict.keys())[list(featureDict.values()).index(i)]),csvData[i]]
        
    
    return (csvData, anomalyLog)
(csvData, anomalyLog)= loadCSV()



[                                        -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [107]:
print(csvData[730][1]) #228 is all N/A, 114 is numbers from a subset, 371 is whole spread, 870 is mostly empty, 

[['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', 'N/A', '', 'Driving'], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''], ['', '', '', ''],

In [2]:
removeEmptyFeatures(csvData)
print(findFeatureIndex('General info', 'Size', csvData))
print(findFeatureIndex('Colors: Classy Red frame', ' Chromed parts and Black accessories & transfers', csvData))

len(csvData)

-1
-1


841

In [51]:
#Data organizing tools
def isFloat(x):
    try:
        float(x)
        return True
    except ValueError:
        return False
def removeEmptyFeatures(csvData):
    empties=[]
    for i in range(len(csvData)):
        content=0
        for j in range(len(csvData[i][1])):
            if (csvData[i][1][j][2]!='' and csvData[i][1][j][2]!='N/A'):
                content=1
        if (content==0):
            empties.insert(0,i)
    for i in empties:
        del csvData[i]
    return csvData  
def findFeatureIndex(Category, Dimension, csvData): #finds index of given feature or -1 if absent|
    feature=(Category,Dimension)
    for i in range(len(csvData)):
        if csvData[i][0]==feature:
            return i
    return -1
def convert2Float(csvData):
    for i in range(1,len(csvData)):
        for j in range(len(csvData[i][1])):
            if isFloat(csvData[i][1][j][1]):
                csvData[i][1][j][1]=float(csvData[i][1][j][1])
            else:
                csvData[i][1][j][1]=''
    return csvData
def convertDriv(csvData):
    for i in range(1,len(csvData)):
        for j in range(len(csvData[i][1])):
            if csvData[i][1][j][3]=='Driven': #Driven=1
                csvData[i][1][j][3]= 1
            elif csvData[i][1][j][3]=='Driving': #Driving=-1
                csvData[i][1][j][3]=-1
            else: #other=0
                csvData[i][1][j][3]=0        
    return csvData
def isolateNumbers(csvData):
    for i in range(1,len(csvData)):
        for j in range(len(csvData[i][1])):
            csvData[i][1][j]=csvData[i][1][j][1]
    return csvData

In [44]:
newcsv=convertDriv(convert2Float(csvData))

print(newcsv[230][1])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [15]:
print(csvData[1][1][2])

['', '388.6', 'mm', 'Driven']


In [43]:
import copy
csvstor=copy.deepcopy(csvData)

In [45]:
csvData=copy.deepcopy(csvstor)

In [52]:
numData=isolateNumbers(removeEmptyFeatures(convertDriv(convert2Float(csvData))))

In [ ]:
def getStatistics(numData)
    #stats: featurename, #nonnumbners, mean, median, SD, max, min
    stats=np.zeros(len(csvData), 7)
    for i in range(1,len(csvData)):
        numbers=[]
        for j in range(len(csvData[i][1])):
            if csvData[i][1][j]!=''
                numbers.append
                